 # 전통적 시계열 모델링

# 1.환경준비

## (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.metrics import *

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore')
warnings.simplefilter('ignore', ConvergenceWarning)

## (2) 함수 생성 

* 잔차분석

In [ ]:
def residual_diag(residuals, lags = 20) :

    print('* 정규성 검정(> 0.05) : ', round(stats.shapiro(residuals)[1],5))
    print('* 정상성 검정(< 0.05) : ', round(sm.tsa.stattools.adfuller(residuals)[1],5))
    print('* 자기상관성 확인(ACF, PACF)')
    fig,ax = plt.subplots(1,2, figsize = (15,5))
    plot_acf(residuals, lags = lags, ax = ax[0])
    plot_pacf(residuals, lags = lags, ax = ax[1])
    plt.show()

## (3) 데이터 불러오기

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/retail_demand2.csv'
data = pd.read_csv(path, usecols = ['date', 'sales', 'tot_sales', 'comp_sales'])
data = data.loc[data['date']<= '2015-10-31'].reset_index(drop = True)
data.head()

* 변수 설명

    * date : 날짜
    * sales : A유통회사 a 매장 aa상품의 일별 판매량
    * tot_sales : A유통회사 전 매장 aa상품의 일별 판매량
    * comp_sales : a매장 인근 B유통회사(경쟁사) b매장 aa상품의 일별 판매량

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(data['sales'])
plt.grid()
plt.show()

In [ ]:
temp = data[-100:]
plt.figure(figsize = (20,8))
plt.plot(temp['sales'])
plt.grid()
plt.show()

# 2.기본 전처리

## (1) 날짜 인덱스

### 1) 날짜 타입으로 변경하기

In [ ]:
# 데이터프레임의 정보를 살펴 봅시다.
data.info()

* 날짜 요소로 변환하기 : pd.to_datetime( *날짜형식으로 저장된 문자열 변수*   , **format** = )  
* format : 일반적인(쉽게 인식 가능한 형태)는 생략 가능 (예 : yyyy-mm-dd hh:mi:ss)
    * to_datetime : https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
    * format : https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data.info()

### 2) 날짜를 인덱스로 변환하기

In [ ]:
data['DT'] = data['date']
data.set_index('DT', inplace=True)
data.head()

### 3) 날짜단위 지정하기 : freq

* **분석 단위**를 어떻게 가져갈 것인가와 관련이 있습니다.
* 시계열 데이터를 **일정한 시간 간격**으로 만들어 줍니다.
* 인덱스 조회시, 마지막에 있는 **freq** 옵션

* 여기서는 일 단위 데이터이므로 D로 지정

In [ ]:
df = data.asfreq('D')
df.head()

## (2) y 만들기

* 사전 관찰(look-ahead) : 미래의 어떤 사실을 안다는 뜻
* 사전 관찰 문제 : 
    * 데이터를 통해 실제로 알아야 하는 시점보다 더 일찍 미래에 대한 사실을 알게 되는 문제.  
    * 사전관찰 문제가 있는 채로 모델링을 하게 되면, 놀라운 성능의 모델이 만들어짐. --> 그러나 실제로는 불가능한 상황.

* 그래서 y를 만들때 사전관찰문제가 발생되지 않도록 해야 함.
    * 예제는 1일 후의 수요량을 예측하려고 합니다.

* 1일 후 수요량을 예측하려면, y를 어떻게 만들어야 할까요?

In [ ]:
df['y'] = df['sales'].shift(-1)
display(df.head())
display(df.tail())

In [ ]:
# 제일 마지막 행은 삭제
df.dropna(axis = 0, inplace = True)
df.tail()

## (3) 데이터 분할

### 1) x, y 나누기

In [ ]:
target = 'y'

x = df.drop([target, 'date'], axis = 1)
y = df.loc[:, target]

### 2) 시계열 데이터 분할

* 다음의 조건으로 Cross Validation을 수행하겠습니다.
    * 3-fold
    * Validation 기간 30일

* TimeSeriesSplit : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
# validation set size
val_size = 30
nfold = 3

tscv = TimeSeriesSplit(n_splits = nfold, test_size = val_size)
tscv

# 3.모델링1 : ARIMA

## (1) y 값 살펴보기

In [ ]:
len(y)

In [ ]:
train = y[:-30]
val = y[-30:]

In [ ]:
train.tail()

In [ ]:
val.head()

In [ ]:
lags = 20

fig,ax = plt.subplots(1,2, figsize = (15,5))
plot_acf(train, lags = lags, ax = ax[0])
plot_pacf(train, lags = lags, ax = ax[1])
plt.show()

## (2) 모델링 : 초기모델

* p, d, q 값을 어떻게 정해야 할까요?
* AR의 p 차수와 MA q 차수 모두 값이 필요해 보입니다. 일단 1, 1을 지정합시다.

### 1) 학습

* sm.tsa.SARIMAX(train, order=(p,d,q)).fit()
    * 모델 선언시 train이 포함
    * .fit()으로 학습.

In [ ]:
# ARIMA 모델링
model1_1 = sm.tsa.SARIMAX(train, order=(1,0,1)).fit()
model1_2 = sm.tsa.SARIMAX(train, order=(1,1,1)).fit()

### 2) 평가

#### ① 잔차진단
SARIMAX로 만든 모델은 .plot_diagnistics 함수를 통해 잔차 진단을 할 수 있습니다.

* 모델.resid : 잔차를 뽑을 수 있습니다.
* 위에서 만든 함수 residual_diag 를 사용하여 잔차진단을 해 봅시다.

In [ ]:
residuals = model1_2.resid
residual_diag(residuals)

#### ② AIC
* 선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
* 값이 작을 수록 좋은 모델
* 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수
* SARIMAX 모델.aic로 쉽게 통계량을 구할 수 있습니다.

In [ ]:
print('model1 AIC :', model1_1.aic)
print('model2 AIC :', model1_2.aic)

#### ③ Validation

시계열 데이터로 실제값과 예측값에 대해 비교하여 그래프를 그려봅시다.

In [ ]:
pred = model1_2.forecast(30)
mean_absolute_error(val, pred)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(train[-120:], label='train')
plt.plot(pred, label = 'forecast')
plt.plot(val, label = 'val')
plt.legend()
plt.show()

## (3) 하이퍼파라미터 튜닝

실제로 p, d, q를 찾는 과정은 마치 Grid Search 처럼 값을 조금씩 조정해가며  최적의 모델을 찾아가는 과정과 유사합니다.


### 1) 학습

In [ ]:
from itertools import product

* 값의 범위 지정

In [ ]:
# product 함수를 이용하여 값의 조합을 구성
p = [0,1,2,3,4]
q = [0,1,2,3,4]
d = [0,1]
iter = list(product(p,d,q))
iter

In [ ]:
# 
mae, aic = [],[]
for i in iter :
    model_fit = sm.tsa.SARIMAX(train, order=(i[0],i[1],i[2])).fit()
    pred = model_fit.forecast(30)
    mae.append( mean_absolute_error(val, pred))
    aic.append(model_fit.aic)
    print(i)

In [ ]:
result = pd.DataFrame({'params(p,d,q)' : iter, 'mae' : mae, 'aic':aic})

display(result.loc[result['mae'] == result.mae.min()])
display(result.loc[result['aic'] == result.aic.min()])

In [ ]:
model2_1 = sm.tsa.SARIMAX(train, order=(3,1,3)).fit()
model2_2 = sm.tsa.SARIMAX(train, order=(4,1,4)).fit()

### 2) 평가

#### ① 잔차진단

* residual_diag

In [ ]:
residuals = model2_2.resid
residual_diag(residuals)

#### ② AIC
* 선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
* 값이 작을 수록 좋은 모델
* 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수

In [ ]:
print('model2 AIC :', model2_2.aic)

#### ③ Validation

In [ ]:
pred = model2_2.forecast(30)
mean_absolute_error(val, pred)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(train[-120:], label='train')
plt.plot(pred, label = 'forecast')
plt.plot(val, label = 'val')
plt.legend()
plt.show()

## (4) Cross Validation

### 1) 학습

In [ ]:
val_size

In [ ]:
rmse, mae, mape, aic = [],[],[],[]
residuals = []
preds = []
p,d,q = 4,1,4

for train_index, val_index in tscv.split(x):

    # 인덱스로 데이터 분할
    train = y[train_index]
    val = y[val_index]

    # 학습
    model = sm.tsa.SARIMAX(train, order=(p,d,q)).fit()

    # 예측
    pred = model.forecast(val_size)
    preds += list(pred)

    # 잔차 저장
    residuals += list(model.resid)

    # 평가
    rmse.append(mean_squared_error(val, pred, squared = False))
    mae.append(mean_absolute_error(val, pred))
    mape.append(mean_absolute_percentage_error(val, pred))
    aic.append(model.aic)

### 2) 평가

#### ① 잔차진단

In [ ]:
residual_diag(residuals)

#### ② AIC

In [ ]:
np.mean(aic)

#### ③ Validation

In [ ]:
preds = pd.Series(preds, index = y[-90:].index)
print('MAE :', mean_absolute_error(y[-90:], preds))

plt.figure(figsize = (20,8))
plt.plot(y[-200:], label = 'train')
plt.plot(y[-90:], label = 'val')
plt.plot(preds, label = 'predicted')

plt.legend()
plt.grid()
plt.show()

# 4.모델링2 : SARIMA

## (1) 모델링 : 초기모델

### 1) 학습

In [ ]:
train = y[:-30]
val = y[-30:]

In [ ]:
# SARIMA 모델링
model2_0 = sm.tsa.SARIMAX(train, order=(4,1,4), seasonal_order=(1,1,1,7)).fit()

### 2) 평가

#### ① 잔차진단

In [ ]:
residuals = model2_0.resid
residual_diag(residuals)

#### ② AIC

In [ ]:
print('model2_0 AIC :', model2_0.aic)

#### ③ Validation

In [ ]:
pred = model2_0.forecast(30)
mean_absolute_error(val, pred)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(train[-120:], label='train')
plt.plot(pred, label = 'forecast')
plt.plot(val, label = 'val')
plt.legend()
plt.show()

## (2) 하이퍼파라미터 튜닝
오래 걸립니다. 돌려 놓고 커피 한잔 타서 오세요.(약 5~6 분)

### 1) 학습

In [ ]:
P = [1,2,3]
Q = [1,2,3]
D = [1]
mae, aic = [],[]
iter = list(product(P,D,Q))

for i in iter :
    model_fit = sm.tsa.SARIMAX(train, order=(4,1,4), seasonal_order=(i[0],i[1],i[2],7)).fit()
    pred = model_fit.forecast(30)
    mae.append( mean_absolute_error(val, pred))
    aic.append(model_fit.aic)
    print(i)

In [ ]:
result = pd.DataFrame({'params(P,D,Q)' : iter, 'mae' : mae, 'aic':aic})

display(result.loc[result['mae'] == result.mae.min()])
display(result.loc[result['aic'] == result.aic.min()])

In [ ]:
model2_1 = sm.tsa.SARIMAX(train, order=(3,1,3), seasonal_order=(1,1,3,7)).fit()
model2_2 = sm.tsa.SARIMAX(train, order=(3,1,3), seasonal_order=(1,1,1,7)).fit()

### 2) 평가

#### ① 잔차진단

* residual_diag

In [ ]:
residuals = model2_1.resid
residual_diag(residuals)

In [ ]:
residuals = model2_2.resid
residual_diag(residuals)

#### ② AIC
* 선형 모델에서의 적합도와, feature가 과도하게 늘어나는 것을 방지하도록 설계된 통계량이 AIC 입니다.
* 값이 작을 수록 좋은 모델
* 공식 : 𝐴𝐼𝐶=−2 ln⁡(𝐿)+2𝑘 ➡ - 모델의 적합도 + 변수의 갯수

In [ ]:
print('model2_1 AIC :', model2_1.aic)
print('model2_2 AIC :', model2_2.aic)

#### ③ Validation

In [ ]:
pred = model2_1.forecast(30)
mean_absolute_error(val, pred)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(train[-120:], label='train')
plt.plot(pred, label = 'forecast')
plt.plot(val, label = 'val')
plt.legend()
plt.show()

## (3) Cross Validation

### 1) 학습

In [ ]:
rmse, mae, mape, aic = [],[],[],[]
residuals = []
preds = []
p,d,q = 4,1,4
P,D,Q = 1,1,3
m = 7

for train_index, val_index in tscv.split(x):

    # 인덱스로 데이터 분할
    train = y[train_index]
    val = y[val_index]

    # 학습
    model = sm.tsa.SARIMAX(train, order=(p,d,q), seasonal_order = (P,D,Q,m)).fit()

    # 예측
    pred = model.forecast(val_size)
    preds += list(pred)

    # 잔차 저장
    residuals += list(model.resid)

    # 평가
    rmse.append(mean_squared_error(val, pred, squared = False))
    mae.append(mean_absolute_error(val, pred))
    mape.append(mean_absolute_percentage_error(val, pred))
    aic.append(model.aic)

### 2) 평가

#### ① 잔차진단

In [ ]:
residual_diag(residuals)

#### ② AIC

In [ ]:
np.mean(aic)

#### ③ Validation

In [ ]:
preds = pd.Series(preds, index = y[-90:].index)
print('MAE :', mean_absolute_error(y[-90:], preds))

plt.figure(figsize = (20,8))
plt.plot(y[-200:], label = 'train')
plt.plot(y[-90:], label = 'val')
plt.plot(preds, label = 'predicted')

plt.legend()
plt.grid()
plt.show()

# 5.모델링3 : SARIMAX

## (1) 모델링

In [ ]:
x.head()

### 1) 학습

In [ ]:
val_size = 30
x_train, y_train = x[:-val_size], y[:-val_size]
x_val, y_val = x[-val_size:], y[-val_size:]

In [ ]:
x_train.shape, y_train.shape

In [ ]:
model3_1 = sm.tsa.SARIMAX(y_train, order=(4,1,4), seasonal_order=(1,1,3,7), exog=x_train).fit()

### 2) 평가

#### ① 잔차진단

* residual_diag

In [ ]:
residuals = model3_1.resid
residual_diag(residuals)

#### ② AIC

In [ ]:
print('model3_1 AIC :', model3_1.aic)

#### ③ Validation
SARIMAX 모델을 생성하고, 예측할 때는 exog=x_val 옵션이 들어가야 함.

In [ ]:
pred = model3_1.forecast(30,  exog=x_val)
mean_absolute_error(val, pred)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(train[-120:], label='train')
plt.plot(pred, label = 'forecast')
plt.plot(val, label = 'val')
plt.legend()
plt.show()

## (2) Cross Validation

### 1) 학습

In [ ]:
rmse, mae, mape, aic = [],[],[],[]
residuals = []
preds = []
p,d,q = 4,1,4
P,D,Q = 1,1,3
m = 7

for train_index, val_index in tscv.split(x):

    # 인덱스로 데이터 분할
    x_train, y_train = x.iloc[train_index], y[train_index]
    x_val, y_val = x.iloc[val_index], y[val_index]

    # 학습
    model_fit = sm.tsa.SARIMAX(y_train, order=(p,d,q), seasonal_order=(P,D,Q,m), exog=x_train).fit()

    # 예측
    pred = model_fit.forecast(val_size, exog=x_val)
    preds += list(pred)

    # 잔차 저장
    residuals += list(model.resid)

    # 평가
    rmse.append(mean_squared_error(y_val, pred, squared = False))
    mae.append(mean_absolute_error(y_val, pred))
    mape.append(mean_absolute_percentage_error(y_val, pred))
    aic.append(model.aic)

### 2) 평가

#### ① 잔차진단

In [ ]:
residual_diag(residuals)

#### ② AIC

In [ ]:
np.mean(aic)

#### ③ Validation

In [ ]:
preds = pd.Series(preds, index = y[-90:].index)
print('MAE :', mean_absolute_error(y[-90:], preds))

plt.figure(figsize = (20,8))
plt.plot(y[-200:], label = 'train')
plt.plot(y[-90:], label = 'val')
plt.plot(preds, label = 'predicted')

plt.legend()
plt.grid()
plt.show()